In [2]:
from langchain_openai import AzureChatOpenAI
from langchain.schema import HumanMessage
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

llm = AzureChatOpenAI(
    openai_api_key="4b94ea21e11a4d46b7b5c17a1ef08de4", 
    api_version='2023-09-01-preview', 
    azure_endpoint='https://egn-llm-hackathon.openai.azure.com/', 
    deployment_name="gpt-35-turbo"
    )
print(llm)

client=<openai.resources.chat.completions.Completions object at 0x733c197cd250> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x733c197dc350> openai_api_key='4b94ea21e11a4d46b7b5c17a1ef08de4' openai_proxy='' azure_endpoint='https://egn-llm-hackathon.openai.azure.com/' deployment_name='gpt-35-turbo' openai_api_version='2023-09-01-preview' openai_api_type='azure'


In [3]:
from langchain_openai import AzureOpenAIEmbeddings
import os

embeddings_model = AzureOpenAIEmbeddings(
    openai_api_key="4b94ea21e11a4d46b7b5c17a1ef08de4", 
    api_version='2023-09-01-preview', 
    azure_endpoint='https://egn-llm-hackathon.openai.azure.com/')
print(embeddings_model)

client=<openai.resources.embeddings.Embeddings object at 0x733c28bb5f50> async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x733c197f86d0> model='text-embedding-ada-002' dimensions=None deployment=None openai_api_version='2023-09-01-preview' openai_api_base=None openai_api_type='azure' openai_proxy='' embedding_ctx_length=8191 openai_api_key='4b94ea21e11a4d46b7b5c17a1ef08de4' openai_organization=None allowed_special=set() disallowed_special='all' chunk_size=16 max_retries=2 request_timeout=None headers=None tiktoken_enabled=True tiktoken_model_name=None show_progress_bar=False model_kwargs={} skip_empty=False default_headers=None default_query=None retry_min_seconds=4 retry_max_seconds=20 http_client=None azure_endpoint='https://egn-llm-hackathon.openai.azure.com/' azure_ad_token=None azure_ad_token_provider=None validate_base_url=True


In [4]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_experimental.text_splitter import SemanticChunker

In [17]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader('queens_speeches/speeches', glob="**/*.txt")
docs = loader.load()


loader_wiki = DirectoryLoader('queens_speeches/wiki', glob="**/*.txt")
docs_wiki = loader_wiki.load()

In [18]:
for i, doc in enumerate(docs):
    doc.metadata["year"] = int(doc.metadata["source"].split('/')[-1].split('.')[0])#.split('/')[-1].split['.'][0]


In [22]:
#text_splitter = RecursiveCharacterTextSplitter(chunk_size = 100, chunk_overlap = 20, length_function = len, is_separator_regex = False)
#documents = text_splitter.split_documents(docs)
#input_documents = [text_splitter.create_documents([d.page_content]) for d in documents]

text_splitter = SemanticChunker(embeddings_model)
documents = text_splitter.split_documents(docs + docs_wiki)


In [9]:
#test_document = documents[0]
#test_document
#documents = text_splitter.create_documents([d.page_content for d in docs])

In [10]:
#input_documents = [y for x in input_documents for y in x]

In [23]:
vector = FAISS.from_documents(documents, embeddings_model)

In [24]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Besvar nedenstående spørgsmål, udelukkende baseret på denne kontekst:

<kontekst>
{context}
</kontekst>

Spørgsmål: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [25]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [26]:
response = retrieval_chain.invoke({"input": "Hvilket år blev Kronprins Frederiks første datter født?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4096 tokens. However, your messages resulted in 4228 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [29]:
spoergsmaal = [
'I hvilket år holdt Dronning Margrethe en nytårstale, der markerede hendes 40-års regeringsjubilæum?',
'Hvem er Danmarks Dronning i februar 2024?',
'Hvornår blev Kongens første datter født?',
'Hvilke lande nævner Dronningen altid i sine nytårstaler?',
'Udover dansk, hvilket andet sprog benytter Dronning Margrethe undertiden i sine nytårstaler?',
'I hvilken nytårstale reflekterede Dronning Margrethe over tabet af Prins Henrik?',
'Hvem kalder Dronningen for Søens Folk?']

for s in spoergsmaal:
    try:
        response = retrieval_chain.invoke({"input": s})
        print(s, response["answer"])
    except Exception as e:
        print("Fejl:", s, e)

I hvilket år holdt Dronning Margrethe en nytårstale, der markerede hendes 40-års regeringsjubilæum? Dronning Margrethe holdt en nytårstale, der markerede hendes 40-års regeringsjubilæum i 2012.
Hvem er Danmarks Dronning i februar 2024? I februar 2024 vil Danmarks dronning være Dronning Mary.


NameError: name 'e' is not defined